In [12]:
import matplotlib.pyplot as plt
import torch
import torchvision
import os
import torch.nn.functional as F
from collections import Counter
from os.path import exists
import torch.optim as optim
import torch.nn as nn
import numpy as np
import torch
import math
import re

from torchvision import transforms

!pip install torchinfo

from torch import nn

In [2]:
!git clone https://github.com/shreyash-99/ERA1.git

Cloning into 'ERA1'...
remote: Enumerating objects: 368, done.
remote: Counting objects: 100% (368/368), done.
remote: Compressing objects: 100% (357/357), done.
remote: Total 368 (delta 9), reused 366 (delta 7), pack-reused 0
Receiving objects: 100% (368/368), 19.48 MiB | 23.44 MiB/s, done.
Resolving deltas: 100% (9/9), done.


In [3]:
cd ERA1/Session17/Assignment_solution/transformer

/content/ERA1/Session17/Assignment_solution/transformer


In [6]:
from transformer import BERT
from bert_helper import SentencesDataset

In [7]:
# =============================================================================
# #Init
# =============================================================================
print('initializing..')
batch_size = 1024
seq_len = 20
embed_size = 128
inner_ff_size = embed_size * 4
n_heads = 8
n_code = 8
n_vocab = 40000
dropout = 0.1
# n_workers = 12

#optimizer
optim_kwargs = {'lr':1e-4, 'weight_decay':1e-4, 'betas':(.9,.999)}

initializing..


In [9]:
cd ../../SessionCodes

/content/ERA1/Session17/SessionCodes


In [13]:
# =============================================================================
# Input
# =============================================================================
#1) load text
print('loading text...')
pth = 'BERT/training.txt'
sentences = open(pth).read().lower().split('\n')

#2) tokenize sentences (can be done during training, you can also use spacy udpipe)
print('tokenizing sentences...')
special_chars = ',?;.:/*!+-()[]{}"\'&'
sentences = [re.sub(f'[{re.escape(special_chars)}]', ' \g<0> ', s).split(' ') for s in sentences]
sentences = [[w for w in s if len(w)] for s in sentences]

#3) create vocab if not already created
print('creating/loading vocab...')
pth = 'vocab.txt'
if not exists(pth):
    words = [w for s in sentences for w in s]
    vocab = Counter(words).most_common(n_vocab) #keep the N most frequent words
    vocab = [w[0] for w in vocab]
    open(pth, 'w+').write('\n'.join(vocab))
else:
    vocab = open(pth).read().split('\n')

#4) create dataset
print('creating dataset...')
dataset = SentencesDataset(sentences, vocab, seq_len)
# kwargs = {'num_workers':n_workers, 'shuffle':True,  'drop_last':True, 'pin_memory':True, 'batch_size':batch_size}
kwargs = {'shuffle':True,  'drop_last':True, 'pin_memory':True, 'batch_size':batch_size}
data_loader = torch.utils.data.DataLoader(dataset, **kwargs)

loading text...
tokenizing sentences...
creating/loading vocab...
creating dataset...


In [14]:
# =============================================================================
# Model
# =============================================================================
#init model
print('initializing model...')
model = BERT(n_code, n_heads, embed_size, inner_ff_size, len(dataset.vocab), seq_len, dropout)
model = model.cuda()

# =============================================================================
# Optimizer
# =============================================================================
print('initializing optimizer and loss...')
optimizer = optim.Adam(model.parameters(), **optim_kwargs)
loss_model = nn.CrossEntropyLoss(ignore_index=dataset.IGNORE_IDX)


initializing model...
initializing optimizer and loss...


In [18]:
cd ../Assignment_solution/transformer


/content/ERA1/Session17/Assignment_solution/transformer


In [19]:
from bert_helper import get_batch

In [20]:
# Train
# =============================================================================
print('training...')
print_each = 10
model.train()
batch_iter = iter(data_loader)
n_iteration = 1000
for it in range(n_iteration):

    #get batch
    batch, batch_iter = get_batch(data_loader, batch_iter)

    #infer
    masked_input = batch['input']
    masked_target = batch['target']

    masked_input = masked_input.cuda(non_blocking=True)
    masked_target = masked_target.cuda(non_blocking=True)
    output = model(masked_input)

    #compute the cross entropy loss
    output_v = output.view(-1,output.shape[-1])
    target_v = masked_target.view(-1,1).squeeze()
    loss = loss_model(output_v, target_v)

    #compute gradients
    loss.backward()

    #apply gradients
    optimizer.step()

    #print step
    if it % print_each == 0:
        print('it:', it,
              ' | loss', np.round(loss.item(),2),
              ' | Δw:', round(model.embeddings.weight.grad.abs().sum().item(),3))

    #reset gradients
    optimizer.zero_grad()

training...
it: 0  | loss 10.2  | Δw: 1.136
it: 10  | loss 9.55  | Δw: 0.558
it: 20  | loss 9.38  | Δw: 0.326
it: 30  | loss 9.16  | Δw: 0.248
it: 40  | loss 9.04  | Δw: 0.208
it: 50  | loss 8.84  | Δw: 0.184
it: 60  | loss 8.69  | Δw: 0.17
it: 70  | loss 8.56  | Δw: 0.158
it: 80  | loss 8.31  | Δw: 0.146
it: 90  | loss 8.22  | Δw: 0.144
it: 100  | loss 8.04  | Δw: 0.125
it: 110  | loss 7.96  | Δw: 0.116
it: 120  | loss 7.75  | Δw: 0.116
it: 130  | loss 7.65  | Δw: 0.109
it: 140  | loss 7.54  | Δw: 0.113
it: 150  | loss 7.43  | Δw: 0.104
it: 160  | loss 7.35  | Δw: 0.094
it: 170  | loss 7.25  | Δw: 0.091
it: 180  | loss 7.08  | Δw: 0.087
it: 190  | loss 7.01  | Δw: 0.093
it: 200  | loss 6.96  | Δw: 0.087
it: 210  | loss 6.9  | Δw: 0.089
it: 220  | loss 6.87  | Δw: 0.083
it: 230  | loss 6.71  | Δw: 0.082
it: 240  | loss 6.67  | Δw: 0.084
it: 250  | loss 6.65  | Δw: 0.081
it: 260  | loss 6.67  | Δw: 0.08
it: 270  | loss 6.66  | Δw: 0.079
it: 280  | loss 6.57  | Δw: 0.083
it: 290  | loss 

In [21]:
# =============================================================================
# Results analysis
# =============================================================================
print('saving embeddings...')
N = 3000
np.savetxt('values.tsv', np.round(model.embeddings.weight.detach().cpu().numpy()[0:N], 2), delimiter='\t', fmt='%1.2f')
s = [dataset.rvocab[i] for i in range(N)]
open('names.tsv', 'w+').write('\n'.join(s) )


print('end')

saving embeddings...
end
